In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import time
import geopandas as gpd
from shapely.geometry import (LinearRing, LineString, MultiLineString,
                              MultiPoint, MultiPolygon, Point, Polygon)
from multiprocessing import Pool
import os
import re
import matplotlib.pyplot as plt
import mytools
import numpy as np

# 一、计算覆盖空洞生成图层

# 1、-----------------现网覆盖区域计算---------------------------

### 20200707 导入计算覆盖半径的数据

In [ ]:
gongcan_loc_juli = pd.read_csv(open('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/1-修正半径后使用的数据/1_修正半径_2周边建筑修正.csv',encoding='gbk'))

In [ ]:
gongcan_loc_juli.head(1)

In [ ]:
gongcan_loc_juli.columns

In [ ]:
#做扇区
print('做扇区')
gongcan_s = mytools.gisn.add_sectors_df(gongcan_loc_juli,coords=['RRU经度', 'RRU纬度', 'd_height', '方位角', 'distance_use_3'], has_z=False, sec_col='geometry', shape_dict={'beam': 120, 'per_degree': 10})

In [ ]:
gongcan_s['归属区县'] = gongcan_s['归属区县'].fillna('wu')

In [ ]:
gongcan_s.to_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/1-修正半径后使用的数据/修正后覆盖半径的扇区图层_1原始.data')

In [ ]:
gongcan_s = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/1-修正半径后使用的数据/修正后覆盖半径的扇区图层_1原始.data')

In [ ]:
gongcan_s.crs

In [ ]:
gongcan_s.plot(figsize=(11,11))

## 去掉农村覆盖到场景边界内的区域

In [ ]:
#---------------------------规划图层缩小50%的距离---------------------------

In [ ]:
guihua = gpd.read_file('G:/1-规划/图层/全省区域规划图层20200110/全省区域规划图层20200110.TAB',encoding='gbk')

In [ ]:
guihua['区域类型2'] = guihua['区域类型'].replace({'3A景区':'农村','4A景区':'农村','5A景区':'农村','景区':'农村','热点':'农村'})

In [ ]:
guihua_use = guihua.loc[guihua['区域类型2']!='农村']

In [ ]:
guihua_yiban = mytools.gisn.changjing_distance(guihua_use,'区域类型2','区域类型')

In [ ]:
guihua_yiban['distance2'] = 0-guihua_yiban['distance']/2

In [ ]:
guihua_res = mytools.gisn.buffer_df(guihua_yiban,'distance2')

In [ ]:
guihua_res.columns =['city', 'region', 'name', 'type', 'area', 'geometry', 'type2', 'distance','distance2']

In [ ]:
guihua_res.to_file('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/2-规划图层缩小场景1800覆盖半径的一半距离/规划图层剔除农村1800场景一半.shp',encoding='utf-8')

In [ ]:
guihua_res= gpd.read_file('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/2-规划图层缩小场景1800覆盖半径的一半距离/规划图层剔除农村1800场景一半.shp',encoding='utf-8')

In [ ]:
guihua_res.plot(figsize=[11,11])

In [ ]:
#---------------------------农村覆盖区域---------------------------

In [ ]:
gongcan_s_nc = gongcan_s.loc[gongcan_s['区域类型']=='农村']

In [ ]:
gongcan_s_nc['所属地区'] = gongcan_s_nc['所属地区'].fillna('wu')

In [ ]:
#全网农村小区覆盖按照地市合并
gongcan_s_nc_city = gongcan_s_nc.dissolve(by = '所属地区')

In [ ]:
#只是留下geo去除索引，索引是地市
gongcan_s_nc_city_use = gongcan_s_nc_city[['geometry']].reset_index()

In [ ]:
gongcan_s_nc_city_use.columns=['city','geometry']

In [ ]:
#在全网农村覆盖区域中去掉农村覆盖到其他区域的
nongcun = gpd.overlay(gongcan_s_nc_city_use,guihua_res,how='difference')

In [ ]:
#农村区域按照地市合并
nongcun_city = nongcun.dissolve(by = 'city')

In [ ]:
nongcun_city =nongcun_city.reset_index()
nongcun_city.to_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/3-小区覆盖区域/农村场景小区覆盖区域.data')

In [ ]:
nongcun_city = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/3-小区覆盖区域/农村场景小区覆盖区域.data')

In [ ]:
nongcun_city.plot(figsize=[11,11])

# 2、-----------------求原始覆盖空洞---------------------------

### 1-按照区县合并覆盖区域

In [ ]:
gongcan_s_feinc = gongcan_s.loc[gongcan_s['区域类型']!='农村']

In [ ]:
#小区覆盖区域图层只保留两列
gongcan_s_feinc = gongcan_s_feinc[['所属地区','geometry']]

In [ ]:
gongcan_s_feinc.columns = ['city','geometry']

In [ ]:
gongcan_s_z = gongcan_s_feinc.append(nongcun_city)

In [ ]:
#小区覆盖区域按照地市合并
gongcan_s_z = gongcan_s_z.dissolve(by = 'city')

In [ ]:
gongcan_s_z.to_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/3-小区覆盖区域/农村和非农村合并全网覆盖图层按照地市合并.data')

In [ ]:
gongcan_s_z.to_file('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/3-小区覆盖区域/农村和非农村合并全网覆盖图层按照地市合并.shp',encoding='utf-8')

In [ ]:
mytools.sound()

### 2-求出覆盖空洞原始

In [ ]:
tu = mytools.gisn.maps()

In [ ]:
city_tu_diff = gpd.overlay(tu,gongcan_s_z,how='difference')

In [ ]:
city_tu_diff.to_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/5-原始空洞/原始空洞.data')

In [ ]:
city_tu_diff.to_file('G:/1-规划/评估/覆盖空洞/20200712第四版本/4-相关图层输出/1原始空洞/原始空洞.shp',encoding='utf-8')

In [ ]:
city_tu_diff.plot(figsize=(11,11),color='gray')

# 3、-------------剔除水系-------------

### 1-图中去湖泊和水系

In [ ]:
hb_sx_city=pd.read_pickle('G:/1-规划/图层/水域/水域按照地市划分.data')

In [ ]:
hb_sx_city.head(1)

In [ ]:
hb_sx_city1 = hb_sx_city.dissolve(by = 'city')

In [ ]:
hb_sx_city1

In [ ]:
hb_sx_city1.to_pickle('G:/1-规划/图层/水域/水域按照地市合并.data')

In [ ]:
mytools.sound()

In [ ]:
kongdong = gpd.overlay(city_tu_diff,hb_sx_city1,how='difference')

In [ ]:
mytools.sound()

In [ ]:
kongdong.to_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/6-剔除水系后空洞/剔除水系后的空洞.data')

In [ ]:
kongdong.to_file('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/6-剔除水系后空洞/剔除水系后的空洞.shp',encoding='utf-8')

In [ ]:
kongdong.plot(figsize=(11,11),color='coral')

In [ ]:
mytools.sound()

# 4、-------------空洞有效数据整理-------------

### 1-拆分成多个空洞-打标场景 划分大中小微

In [ ]:
kongdong.head(1)

In [ ]:
kongdong['type'] =  kongdong.type

In [ ]:
kongdong.groupby('type').count()

In [ ]:
kongd_use = kongdong.loc[kongdong['type']=='Polygon'].append(kongdong.loc[kongdong['type']=='MultiPolygon'].explode())

In [ ]:
kongd_use.shape

In [ ]:
kongd_mianji = mytools.gisn.add_area(kongd_use)

In [ ]:
kongd_mianji['type'] =  kongd_mianji.type

In [ ]:
kongd_mianji =kongd_mianji.reset_index(drop=True)

In [ ]:
# 打标场景

In [ ]:
kongd_mianji[['lon','lat']] = kongd_mianji.apply(lambda x:pd.Series(x['geometry'].centroid.coords[:][0]),axis=1)#

In [ ]:
result = mytools.gisn.points_coverage_merge(kongd_mianji,'lon','lat')

In [ ]:
result.shape

In [ ]:
#添加id编号
result_id = result.reset_index(drop=True)
result['id'] = [ 'kd_{}'.format(str(name)) for name in result_id.index+1]

In [ ]:
result['制式'] = 'FDD1800'
result.loc[result['区域类型']=='农村','制式']='FDD900'

In [ ]:
result = mytools.gisn.changjing_distance(result,'区域类型','制式')

In [ ]:
result['标准面积'] = np.pi* result['distance']* result['distance']
result['空洞面积比例'] = result['面积']/result['标准面积']

In [ ]:
def fuck_data(x):
    if x['空洞面积比例']>=1:
        return '大'
    elif x['空洞面积比例']>=1/3:
        return '中'
    elif x['空洞面积比例']>=1/6:
        return '小'
    else:
        return '微'

In [ ]:
result['空洞级别'] = result.apply(fuck_data,axis=1)

In [ ]:
result.head(1)

In [ ]:
result.to_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/3-拆分后的空洞/覆盖空19481个添加场景区分大中小微.data')

In [ ]:
result.reindex(columns=['id','city','面积','lon','lat', '区域类型','制式','distance','标准面积','空洞面积比例','空洞级别']).to_excel(
    'G:/1-规划/评估/覆盖空洞/20200712第四版本/3-拆分后的空洞/覆盖空19481个添加场景区分大中小微.xlsx',index=False)

In [ ]:
result.columns = ['city', 'type', 'geometry', 'area', 'lon', 'lat', 'regiontype', 'id', 'frequency ',
       'distance', 'scene_area', 'kd_area_specific', 'kd_class']

In [ ]:
result.to_file('G:/1-规划/评估/覆盖空洞/20200712第四版本/3-拆分后的空洞/覆盖空19481个添加场景区分大中小微.shp',encoding='utf-8')

In [ ]:
result.groupby('kd_class').count()